In [2]:
#Dash
import numpy as np
import pandas as pd
from sklearn.manifold import MDS
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import json

# Step 1: 車站經緯度
coordinates = {
    'Taipei': (121.5170416140462, 25.048144340471694),
    'Hsinchu': (120.97218465041708, 24.803620504236306),
    'Taichung': (120.68502265782085, 24.136965842698647),
    'Douliu': (120.54100869869137, 23.71194174320373),
    'Kaohsiung': (120.30263663949066, 22.63967061761657),
    'Hualien': (121.31177017259222, 23.331693127455807),
    'Taitung': (121.06074805573594, 22.710408169564232)
}

cities = list(coordinates.keys())
coords = np.array(list(coordinates.values()))

# Step 2: 距離矩陣計算
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0 # Earth radius in kilometers
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    distance = R * c
    return distance

n = len(coords)
distance_matrix = np.zeros((n, n))

for i in range(n):
    for j in range(i + 1, n):
        distance = haversine(coords[i][1], coords[i][0], coords[j][1], coords[j][0])
        distance_matrix[i, j] = distance
        distance_matrix[j, i] = distance

# 印出距離矩陣
# Convert distance matrix to DataFrame
distance_df = pd.DataFrame(distance_matrix, index=cities, columns=cities)

max_city_name_length = max(len(city) for city in cities)
print("Distance Matrix:")
# Print header row
header = " " * (max_city_name_length + 5)  # Add extra space for alignment
for city in cities:
    header += f"{city:>{max_city_name_length}}"
print(header)

# Print distance values
for i, city in enumerate(cities):
    row_str = f"{city:<{max_city_name_length + 5}}"  # Add extra space for alignment
    for j in range(len(cities)):
        row_str += f"{distance_df.iloc[i, j]:>{max_city_name_length}.2f}"
    print(row_str)

# Step 3: MDS
mds = MDS(n_components=2, dissimilarity="precomputed", random_state=42, normalized_stress='auto')
mds_coords = mds.fit_transform(distance_matrix)

# Step 4: 畫圖
app = dash.Dash(__name__)

colors = ['red', 'green', 'blue', 'cyan', 'magenta', 'yellow', 'purple']
city_colors = dict(zip(cities, colors))

data = [
    go.Scatter(
        x=mds_coords[:, 1],
        y=mds_coords[:, 0],
        mode='markers+text',
        text=cities,
        marker=dict(color=[city_colors[city] for city in cities], size=12),
        textposition='top center'
    )
]

layout = go.Layout(
    title='MDS of Train Stations on Geographical Distance',
    xaxis=dict(title='X'),
    yaxis=dict(title='Y'),
    hovermode='closest'
)

app.layout = html.Div([
    dcc.Graph(
        id='mds-graph',
        figure=dict(data=data, layout=layout)
    ),
    html.Div([
        dcc.Markdown("""
            **Hover Data**
            Mouse over values in the graph.
        """),
        html.Pre(id='hover-data')
    ]),
    html.Div([
        dcc.Markdown("""
            **Click Data**
            Click on points in the graph.
        """),
        html.Pre(id='click-data'),
    ]),
    html.Div([
        dcc.Markdown("""
            **Selection Data**
            Choose the lasso or rectangle tool in the graph's menu
            bar and then select points in the graph.
        """),
        html.Pre(id='selected-data'),
    ])
])

@app.callback(
    Output('hover-data', 'children'),
    [Input('mds-graph', 'hoverData')]
)
def display_hover_data(hoverData):
    return json.dumps(hoverData, indent=2)

@app.callback(
    Output('click-data', 'children'),
    [Input('mds-graph', 'clickData')]
)
def display_click_data(clickData):
    return json.dumps(clickData, indent=2)

@app.callback(
    Output('selected-data', 'children'),
    [Input('mds-graph', 'selectedData')]
)
def display_selected_data(selectedData):
    return json.dumps(selectedData, indent=2)

if __name__ == '__main__':
    app.run_server(debug=True)


Distance Matrix:
                 Taipei  Hsinchu Taichung   DouliuKaohsiung  Hualien  Taitung
Taipei             0.00    61.30   131.69   178.46   294.91   191.99   264.05
Hsinchu           61.30     0.00    79.62   129.02   250.09   167.26   232.93
Taichung         131.69    79.62     0.00    49.48   171.00   109.94   163.19
Douliu           178.46   129.02    49.48     0.00   121.70    89.24   123.38
Kaohsiung        294.91   250.09   171.00   121.70     0.00   128.81    78.18
Hualien          191.99   167.26   109.94    89.24   128.81     0.00    73.71
Taitung          264.05   232.93   163.19   123.38    78.18    73.71     0.00
